In [1]:
import pandas as pd
import os
import re
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
fin_train = pd.read_csv("pipeline/argumentation-based/data/train.csv")
fin_val = pd.read_csv("pipeline/argumentation-based/data/validation.csv")
fin_test = pd.read_csv("pipeline/argumentation-based/data/test.csv")

df = pd.concat([fin_train, fin_val, fin_test])
display(df.head(5))

,ID,text,claim,evidence,structure,label,dataset
0,34015,this ad is disgusting in so many ways. the vid...,promotes bullying of kids by other kids suppor...,but at a time when schools and parents are bei...,claims:promotes bullying of kids by other kids...,FAKE,kaggle
1,37654,in a press conference on thursday president tr...,the news would actually be more interesting to...,a segment after the press conference cnns jake...,claims:the news would actually be more interes...,FAKE,kaggle
2,3864,australia to hunt down anti vax nurses and pro...,australia to hunt down anti vax nurses and pro...,countries like australia demonize other nation...,claims:australia to hunt down anti vax nurses ...,FAKE,fake_real
3,44212,our weekly documentary screening curated by th...,<unk>,our weekly documentary screening curated by th...,claims:<unk>evidences:our weekly documentary s...,FAKE,kaggle
4,1686,says most of perrys chiefs of staff have been ...,<unk>,<unk>,claims:<unk>evidences:<unk>,REAL,liar


In [3]:
grouped = df.groupby('dataset')
for i in grouped:
    print(i[0])
    print(len(i[1]))

fake_real
1483
kaggle
3676
liar
2003


In [4]:
def n_label(df, label):
    """ Returns number of examples with a specific label """
    return len(df[df["label"] == label])

def plot_stats(df):
    result = pd.Series(
        data={
        "Total": df.shape[0], 
        "# Fake": n_label(df, "FAKE"),
        "# Real": n_label(df, "REAL"),
        "% Fake": "{:.2f}".format(n_label(df, "FAKE") / len(df))
        }
    )
    display(result)

### MARGOT

In [ ]:
# def rejoin_data(original, 
#                 structure) -> dict[pd.DataFrame]:

#     merged_df = pd.concat([original.reset_index(drop=False), structure.reset_index(drop=True)], axis=1).dropna()
#     # merged_df = merged_df.loc[(structure['claim'] != "[]") | (structure['evidence'] != "[]")]
#     merged_df = merged_df.set_index('ID')

#     merged_df['claim'] = merged_df["claim"].apply(lambda x: list(eval(x)))
#     merged_df["evidence"] = merged_df["evidence"].apply(lambda x: list(eval(x)))

#     # #Merge claim & evidence together as "structure"
#     merged_df["claim"] = merged_df["claim"].apply(lambda x: ', '.join(x) if x else '<unk>')
#     merged_df["evidence"] = merged_df["evidence"].apply(lambda x: ', '.join(x) if x else '<unk>')
#     claims = "claims:" + merged_df["claim"].values
#     evidences = "evidences:" + merged_df["evidence"].values

#     merged_df["structure"] = claims + evidences
#     # merged_df.dropna(inplace=True)
#     return merged_df

# # Path to batches, without 
# path = "pipeline/argumentation-based/argumentation structure/argumentation structure fin/margot"

# path_to_clean_data = "/Volumes/CentrumZijnsorientatie/Thesis/fake_news/pipeline/argumentation-based/data"


# structure = pd.read_csv(f"{path}/kaggle/validation_batch_100.csv", index_col='ID')
# original = pd.read_csv(f"{path_to_clean_data}/kaggle/validation.csv", index_col='ID')
# rejoined = rejoin_data(original, structure)
# rejoined.to_csv(f"{path}/kaggle/validation.csv", index_label="ID", columns=['text', 'claim', 'evidence', 'structure', 'label'])


In [ ]:
def load_data(path):
    data = {}
    for f in os.listdir(path):
        name = f.split('.')[0]
        df = pd.read_csv(f"{path}/{f}", usecols=['ID', 'text', 'claim', 'evidence', 'structure', 'label'])
        df = df.reset_index(drop=True).set_index('ID')
        data[name] = df
    return data


In [ ]:
annotated_0 = load_data("pipeline/argumentation-based/argumentation structure/argumentation structure fin/margot")

display(annotated_0['fake_real'].shape) 
display(annotated_0['kaggle'].shape)
display(annotated_0['liar'].shape)

# TO DO UNNEST, only train

In [ ]:
annotated_1 = load_data("pipeline/argumentation-based/argumentation structure/argumentation structure fin/margot_1000")

display(annotated_1['fake_real'].shape) 
display(annotated_1['kaggle'].shape)
display(annotated_1['liar'].shape)


In [ ]:
res = {}
for i in annotated_0:
    merged = pd.concat([annotated_0[i], annotated_1[i]])
    print("# Before removing duplicates and adding col")
    print(merged.shape) 
    merged.drop_duplicates(inplace=True)
    merged['dataset'] = i
    res[i] = merged
    print(merged.shape)
    display(merged.head(5))
    print()    

In [ ]:
df_final = pd.concat([res['fake_real'], res['kaggle'], res['liar']])
display(df_final)

In [ ]:
plot_stats(df_final)

In [ ]:
train, validation = train_test_split(df_final, test_size=0.2)
validation, test = train_test_split(validation, test_size=0.25)

display(train.shape, validation.shape, test.shape)

In [ ]:
plot_stats(train)
plot_stats(validation)
plot_stats(test)

In [ ]:
train.to_csv('pipeline/argumentation-based/argumentation structure/FINAL/train.csv', index='ID')
validation.to_csv('pipeline/argumentation-based/argumentation structure/FINAL/validation.csv', index='ID')
test.to_csv('pipeline/argumentation-based/argumentation structure/FINAL/test.csv', index='ID')

## Dolly

In [ ]:
def combine_batches(batches, name, path) -> None:
    sorted_files = sorted(batches, key=lambda x: int(re.findall(r'\d+', x)[0]))

    idx, batched = [], []
    for f in sorted_files: 
        idx.append(int(re.findall(r'\d+', f)[0]))
        batched.append(pd.read_csv(f"{PATH}/{f}"))

    res = pd.concat(batched, ignore_index=True)
    res.set_index("ID", inplace=True)

    res.to_csv(f"{path}/{name}_batch_{idx[0]}-{idx[-1]}.csv")

    for f in sorted_files:
        os.remove(f"{path}/{f}")

PATH = "pipeline/argumentation-based/argumentation structure/argumentation structure fin/dolly/liar"

# batches_train, batches_test, batches_val = [], [], []

# for f in os.listdir(PATH):
#     if f.startswith("train_") and not re.match(r'\w+_\w+_\d+-\d+', f):
#         batches_train.append(f)
#     if f.startswith("validation_") and not re.match(r'\w+_\w+_\d+-\d+', f):
#         batches_val.append(f)
#     if f.startswith("test_") and not re.match(r'\w+_\w+_\d+-\d+', f):
#         batches_test.append(f)


# if len(batches_train) > 1:
#     combine_batches(batches_train, "train", PATH)

# if len(batches_test) > 1:
#     combine_batches(batches_test, "test")

In [ ]:
path = "pipeline/argumentation-based/argumentation structure/argumentation structure fin/dolly"

res = load_data(path)


In [ ]:
for i in res:
    df = res[i]
    df.drop_duplicates(inplace=True)
    df['dataset'] = i
    df['label'] = df['label'].apply(lambda x: 'FAKE' if x == 0 else ('REAL' if x == 1 else x))
    res[i] = df
    plot_stats(df)
    display(df.head(5))


In [ ]:
df_final = pd.concat([res['fake_real'], res['kaggle'], res['liar']])
display(df_final)

In [ ]:
train, validation = train_test_split(df_final, test_size=0.2)
validation, test = train_test_split(validation, test_size=0.25)

display(train.shape, validation.shape, test.shape)

In [ ]:
train.to_csv('pipeline/argumentation-based/argumentation structure/FINAL/dolly/train.csv', index='ID')
validation.to_csv('pipeline/argumentation-based/argumentation structure/FINAL/dolly/validation.csv', index='ID')
test.to_csv('pipeline/argumentation-based/argumentation structure/FINAL/dolly/test.csv', index='ID')

In [ ]:
df_final.head(5)

In [ ]:
df_final.shape

In [15]:
DATASET = "fake_real"
path_to_clean = f"/Volumes/CentrumZijnsorientatie/Thesis/fake_news/pipeline/argumentation-based/data"

res = {}

for d in os.listdir(path_to_clean):
    tmp = {}

    for f in os.listdir(f"{path_to_clean}/{d}"):
        name = f.split('.')[0]
        # for t in os.listdir(f"{path_to_clean}/{d}/{f}"):
            # print(t)

        df = pd.read_csv(f"{path_to_clean}/{d}/{f}", index_col="ID")
        # index = df_final[df_final['dataset'] == d]
        tmp[name] = df

    res[d] = tmp

# display(res['kaggle']['train'])
fr = pd.concat([res['fake_real']['train'], res['fake_real']['validation'], res['fake_real']['test']])
fr = fr.loc[df_final[df_final['dataset'] == 'fake_real'].index, :] 


kaggle = pd.concat([res['kaggle']['train'], res['kaggle']['validation'], res['kaggle']['test']])
kaggle = kaggle.loc[df_final[df_final['dataset'] == 'kaggle'].index, :] 

liar = pd.concat([res['liar']['train'], res['liar']['validation'], res['liar']['test']])
liar = liar.loc[df_final[df_final['dataset'] == 'liar'].index, :] 

df = pd.concat([fr, kaggle, liar])
# df.drop_duplicates(inplace=True, keep='first')
# print(df.shape)
# display(df)
# df = df.loc[df_final.index, :]  # works
# df.drop_duplicates(inplace=True)

print(df.shape)
# for i in res:
#     for j in res[i]:
        
# print(full_data['train'])

NotADirectoryError: [Errno 20] Not a directory: '/Volumes/CentrumZijnsorientatie/Thesis/fake_news/pipeline/argumentation-based/data/.DS_Store'

In [ ]:
df.head()

In [ ]:
df.to_csv("pipeline/argumentation-based/argumentation structure/FINAL/clean.csv", index_label='ID')

In [ ]:
train, validation = train_test_split(df, test_size=0.2)
validation, test = train_test_split(validation, test_size=0.25)

display(train.shape, validation.shape, test.shape)

In [16]:
train.to_csv('pipeline/argumentation-based/data/train.csv', index='ID')
validation.to_csv('pipeline/argumentation-based/data/validation.csv', index='ID')
test.to_csv('pipeline/argumentation-based/data/test.csv', index='ID')

NameError: name 'train' is not defined